## Part 2 


#### Dictionary containing the occurrences of each annotation in the training set

In [1]:
annotationDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
with open("train", encoding="utf-8") as file:
    my_list = file.readlines()
    my_list = [x.strip() for x in my_list]
    for i in range(len(my_list)):
        annotated_word = my_list[i]
        each_word = annotated_word.split(" ")
        if (len(each_word) == 2):
            if each_word[1] == "O":
                annotationDict["O"] += 1
            elif each_word[1] == "I-positive":
                annotationDict["I-positive"] += 1
            elif each_word[1] == "B-positive":
                annotationDict["B-positive"] += 1
            elif each_word[1] == "I-negative":
                annotationDict["I-negative"] += 1
            elif each_word[1] == "B-negative":
                annotationDict["B-negative"] += 1
            elif each_word[1] == "I-neutral":
                annotationDict["I-neutral"] += 1
            elif each_word[1] == "B-neutral":
                annotationDict["B-neutral"] += 1

    print(annotationDict)

{'O': 91753, 'I-positive': 1653, 'B-positive': 2613, 'I-negative': 443, 'B-negative': 1299, 'I-neutral': 5272, 'B-neutral': 5722}


#### (a) Write a function that estimates the emission parameters from the training set using MLE

In [2]:
def estimate_emission(word, tag):
    with open("train", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
        countTop = 0
        countBottom = annotationDict[tag]
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if len(each_word) == 2:
                if (each_word[0] == word and each_word[1] == tag):
                    countTop += 1
                    
#         print(countTop)
#         print(countBottom)
        return countTop/countBottom

estimate_emission("service", "O")

0.0001198870881606051

#### (b) Replace the words that appear less than k times in the training set with a special token #UNK# before training. This leads to a “modified training set”.

In [3]:
import datetime

def modify_trainingset(k):
    with open("train", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
        wordCountDict = {}
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in wordCountDict:
                wordCountDict[each_word[0]] += 1
            else:
                wordCountDict[each_word[0]] = 1
        
       
        wordToBeReplacedDict = {}
        for key in wordCountDict:
            if wordCountDict[key] < k:
                wordToBeReplacedDict[key] = wordCountDict[key]

        start = datetime.datetime.now()
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            for key in wordToBeReplacedDict:
                if each_word[0] == key:
                    my_list[i] = "#UNK# " + each_word[1]

        modifiedTotalTrainSet = "\n".join(my_list)

        f = open("modified_train","w+")
        f.write(modifiedTotalTrainSet)
                            
        end = datetime.datetime.now()
        elapsed = end - start
        print(elapsed)
            

    
modify_trainingset(3)


0:04:42.321908


In [4]:
def get_word_dict():
    with open("modified_train", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
        modifiedWordDict = {}        
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if each_word[0] in modifiedWordDict:
                modifiedWordDict[each_word[0]] += 1
            else:
                modifiedWordDict[each_word[0]] = 1
            
        return modifiedWordDict
    
modifiedWordDict = get_word_dict()

In [5]:
def estimate_emission_fix(word, tag):
    with open("modified_train", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
        countTop = 0
        countBottom = annotationDict[tag]
        if word in modifiedWordDict:  
            # do nothing
            word = word
            
        else:
            word = "#UNK#"
            
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if (each_word[0] == word and each_word[1] == tag):
                countTop += 1
        
        return countTop/countBottom

estimate_emission_fix("service", "O")

0.0001198870881606051

#### (c) Implement a simple sentiment analysis system that produces the tag

In [6]:
def sentiment_analysis(word):
    OScore = estimate_emission_fix(word, "O")
    BPlusScore = estimate_emission_fix(word, "B-positive")
    IPlusScore = estimate_emission_fix(word, "I-positive")
    BMinusScore = estimate_emission_fix(word, "B-negative")
    IMinusScore = estimate_emission_fix(word, "I-negative")
    BNeutralScore = estimate_emission_fix(word, "B-neutral")
    INeutralScore = estimate_emission_fix(word, "I-neutral")

    scoreDict = {"O": OScore, "I-positive": IPlusScore, "B-positive": BPlusScore, "I-negative":IMinusScore, "B-negative": BMinusScore, "I-neutral":INeutralScore, "B-neutral":BNeutralScore}
    maximumScore = max(scoreDict.values())
    return max(scoreDict, key=scoreDict.get)

   
sentiment_analysis("service")

'I-positive'

In [7]:
import datetime
def predict_sentiment():
    wordDict = {}
    with open("dev.in", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]

        f = open("dev.final.out","w+")
        start = datetime.datetime.now()
        for i in range(len(my_list)):
            each_word = my_list[i]
            if (i > 4300):
                print(". ",end="")
            if each_word in wordDict:
                tag = wordDict[each_word]
                f.write(each_word +  " " + tag + "\n")
            else:
                if each_word != "":
                    tag = sentiment_analysis(each_word)
                    wordDict[each_word] = tag 
                    f.write(each_word +  " " + tag + "\n")
                else:
                    f.write(" \n")

        end = datetime.datetime.now()
        elapsed = end - start
        print(elapsed)

predict_sentiment()

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1:03:09.561558


#### Calculate F score, use evalScript

16767


### Part 3 
#### Function to estimate Transition Parameters using MLE

In [92]:
def estimate_transition(current_tag, previous_tag):
    with open("modified_train_final2", encoding="utf-8") as file:
        my_list = file.readlines()
        my_list = [x.strip() for x in my_list]
#         because last sentence has no ''
        countStart = 1
        countBottom = 0
        countTop = 0
        # counting starts
        giant_each_tweet = []
        print(len(my_list))
        each_tweet = []
        for i in range(len(my_list)):
            annotated_word = my_list[i]
            each_word = annotated_word.split(" ")
            if each_word == ['']:
                countStart += 1
                giant_each_tweet.append(each_tweet)
                each_tweet = []
                
            else:
                each_tweet.append(each_word[1])
                
#         print(giant_each_tweet)
        startDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
        for i in range(len(giant_each_tweet)):
            each_tweet = giant_each_tweet[i]
            if each_tweet[0] == "O":
                startDict["O"] += 1
            elif each_tweet[0] == "I-positive":
                startDict["I-positive"] += 1
            elif each_tweet[0] == "B-positive":
                startDict["B-positive"] += 1
            elif each_tweet[0] == "I-negative":
                startDict["I-negative"] += 1
            elif each_tweet[0] == "B-negative":
                startDict["B-negative"] += 1
            elif each_tweet[0] == "I-neutral":
                startDict["I-neutral"] += 1
            elif each_tweet[0] == "B-neutral":
                startDict["B-neutral"] += 1
                    
        stopDict = {"O": 0, "I-positive": 0, "B-positive": 0, "I-negative":0, "B-negative": 0, "I-neutral":0, "B-neutral":0}
        for i in range(len(giant_each_tweet)):
            each_tweet = giant_each_tweet[i]
            last_index = len(each_tweet) - 1
            if each_tweet[last_index] == "O":
                stopDict["O"] += 1
            elif each_tweet[last_index] == "I-positive":
                stopDict["I-positive"] += 1
            elif each_tweet[last_index] == "B-positive":
                stopDict["B-positive"] += 1
            elif each_tweet[last_index] == "I-negative":
                stopDict["I-negative"] += 1
            elif each_tweet[last_index] == "B-negative":
                stopDict["B-negative"] += 1
            elif each_tweet[last_index] == "I-neutral":
                stopDict["I-neutral"] += 1
            elif each_tweet[last_index] == "B-neutral":
                stopDict["B-neutral"] += 1
                
        if previous_tag == "START":
            # current tag is Yn
            countTop = startDict[current_tag]
            return countTop/countStart
                
        elif current_tag == "STOP":
            # previous tag is Yn
            countTop = stopDict[previous_tag]
            countBottom = annotationDict[previous_tag]
            return countTop/countBottom
            
        else:
            giantTransitionDict = {}
            for i in range(len(giant_each_tweet)):
                each_tweet = giant_each_tweet[i]
                for j in range(len(each_tweet)):
                    if j + 1 == len(each_tweet):
                        break
                    if each_tweet[j] == "O" and each_tweet[j+1] ==current_tag:
                        if ("O"+current_tag) in giantTransitionDict:
                            giantTransitionDict["O"+current_tag] += 1
                        else:
                            giantTransitionDict["O"+current_tag] = 1
                            
                    elif each_tweet[j] == "B-negative" and each_tweet[j+1] ==current_tag:
                        if ("B-negative"+current_tag) in giantTransitionDict:
                            giantTransitionDict["B-negative"+current_tag] += 1
                        else:
                            giantTransitionDict["B-negative"+current_tag] = 1
                            
                    elif each_tweet[j] == "B-positive" and each_tweet[j+1] ==current_tag:
                        if ("B-positive"+current_tag) in giantTransitionDict:
                            giantTransitionDict["B-positive"+current_tag] += 1
                        else:
                            giantTransitionDict["B-positive"+current_tag] = 1
                            
                    elif each_tweet[j] == "B-neutral" and each_tweet[j+1] ==current_tag:
                        if ("B-neutral"+current_tag) in giantTransitionDict:
                            giantTransitionDict["B-neutral"+current_tag] += 1
                        else:
                            giantTransitionDict["B-neutral"+current_tag] = 1
                            
                    elif each_tweet[j] == "I-positive" and each_tweet[j+1] ==current_tag:
                        if ("I-positive"+current_tag) in giantTransitionDict:
                            giantTransitionDict["I-positive"+current_tag] += 1
                        else:
                            giantTransitionDict["I-positive"+current_tag] = 1
                            
                    elif each_tweet[j] == "I-negative" and each_tweet[j+1] ==current_tag:
                        if ("I-negative"+current_tag) in giantTransitionDict:
                            giantTransitionDict["I-negative"+current_tag] += 1
                        else:
                            giantTransitionDict["I-negative"+current_tag] = 1
                            
                    elif each_tweet[j] == "I-neutral" and each_tweet[j+1] ==current_tag:
                        if ("I-neutral"+current_tag) in giantTransitionDict:
                            giantTransitionDict["I-neutral"+current_tag] += 1
                        else:
                            giantTransitionDict["I-neutral"+current_tag] = 1
            print(giantTransitionDict)
            countTop = giantTransitionDict[previous_tag+current_tag]
            countBottom = annotationDict[previous_tag]
            return countTop/countBottom
        
        return 1
# estimate_transition("O", "START")
# ineg = estimate_transition("STOP", "I-negative")
# ipos = estimate_transition("STOP", "I-positive")
# bneg = estimate_transition("STOP", "B-negative")
# bpos = estimate_transition("STOP", "B-positive")
# ineu = estimate_transition("STOP", "I-neutral")
# bneu = estimate_transition("STOP", "B-neutral")
# o = estimate_transition("STOP", "O")
# result = ineg + ipos + bneg + bpos +ineu + bneu + o
# print(result)
estimate_transition("B-neutral", "O")

115855
{'OB-neutral': 5260, 'B-neutralB-neutral': 32, 'I-neutralB-neutral': 17, 'B-positiveB-neutral': 6, 'I-negativeB-neutral': 1, 'I-positiveB-neutral': 1}


0.05732782579316208